# BG3(Baldur's Gate 3) script data extract
- 현재 발더스 게이트3는 유저번역이 진행중입니다. 
- 따라서 두개의 단계로 나뉘게 될겁니다.
- 이미 유저번역이 진행된 데이터는 추출되어 모델의 fine-tuining을 위한 자료로 쓰이게 됩니다. 
- 아직 번역이 되지 않은 부분은 영문장으로 추출되어 번역모델의 훈련이 완료 되었을 때 기계번역 시도를 위한 데이터가 될 것입니다. 

## (1) Xml parser를 통한 script 추출

In [1]:
import xml.etree.ElementTree as ET
import pandas as pd

In [2]:
# xml파일 파싱해서 contestuid를 key로 하고 문자열을 value로 하는 사전을 리턴
# contestuid가 중복이 있을 경우 그에 따른 메세지 도출

def parse_xml_to_dict(script_dir):
    result_dict = {}
    tree = ET.parse(script_dir)
    root = tree.getroot()
    
    # root = ET.fromstring(xml_content)
    for content_elem in root.findall('content'):
        contentuid = content_elem.get('contentuid')
        content_text = content_elem.text
        if result_dict.get(contentuid) == None:
            result_dict[contentuid] = content_text
        else:
            print("Duplicate Key!!", contentuid, content_text)
        
    return result_dict

In [4]:
userTranslate_path = "../train_data/BG3(raw)/ht_ko_en.xml"

In [5]:
userTranslate_dict = parse_xml_to_dict(userTranslate_path)

In [6]:
english_path = "../train_data/BG3(raw)/en.xml"

In [7]:
english_dict = parse_xml_to_dict(english_path)

In [8]:
userTranslate_dict

{'h000006d4gcefbg4092gbb39gfeb27a3bb0a7': "Sorry, darling, I haven't got time for underlings.",
 'h000011feg0be5g4f09g978eg030b3e2e62c6': '주문 교체하기',
 'h00010f77gdae2g459eg9e30g36892aea6b2d': '<LSTag Tooltip="MovementSpeed">이동 속도</LSTag>: [1]/[2]',
 'h00011ac6ge2f4g4104gbc6fg246521189b73': "If that's her 'formula' I can smell, it's even fouler than her blood. Gods below...",
 'h00011aceg6d1bg4430g8e5cg8672138c6d82': '그리고 네 정체가 너가 말한대로가 아니라면 정당한 공격대상으로 간주하겠어.',
 'h0001dc8ag4fd9g4d8ag8775ga6313d6a7ff4': "*Thus you shall spend the rest of your days rutting, the lash of Father's disappointment the only thought in your rat's mind.*",
 'h0001eb83ge642g48eegaceeg497c92c8b8d3': 'Ulma',
 'h00022debgbe37g460cg83a5g28893d382c44': 'We had barely begun to eat when she spoke for the first time. I only caught one word - my name.',
 'h0002ca0fg57a0g4d61g8475gf10d818fe137': 'Spreading Spores',
 'h000347eag76d8g47b6g9fe3g4f2162783f9d': 'That did the trick.',
 'h00039766g9af0g496eg8024g3b3da11cdde5': '*A 

In [9]:
english_dict

{'h000006d4gcefbg4092gbb39gfeb27a3bb0a7': "Sorry, darling, I haven't got time for underlings.",
 'h000011feg0be5g4f09g978eg030b3e2e62c6': 'Replace Spell',
 'h00010f77gdae2g459eg9e30g36892aea6b2d': '<LSTag Tooltip="MovementSpeed">Movement Speed</LSTag>: [1]/[2]',
 'h00011ac6ge2f4g4104gbc6fg246521189b73': "If that's her 'formula' I can smell, it's even fouler than her blood. Gods below...",
 'h00011aceg6d1bg4430g8e5cg8672138c6d82': "And if you ain't who you say you are, I reckon you're fair game.",
 'h0001dc8ag4fd9g4d8ag8775ga6313d6a7ff4': "*Thus you shall spend the rest of your days rutting, the lash of Father's disappointment the only thought in your rat's mind.*",
 'h0001eb83ge642g48eegaceeg497c92c8b8d3': 'Ulma',
 'h00022debgbe37g460cg83a5g28893d382c44': 'We had barely begun to eat when she spoke for the first time. I only caught one word - my name.',
 'h0002ca0fg57a0g4d61g8475gf10d818fe137': 'Spreading Spores',
 'h000347eag76d8g47b6g9fe3g4f2162783f9d': 'That did the trick.',
 'h00039

In [15]:
def detect_language(text):
    total_chars = len(text)

    korean_count = sum(1 for char in text if 0xAC00 <= ord(char) <= 0xD7A3)

    return korean_count / total_chars #>= 0.5

In [16]:
print("문장:", userTranslate_dict["h01306e5egfa1eg4b99gbfc4gc22e75bfd54b"])
print(detect_language(userTranslate_dict["h01306e5egfa1eg4b99gbfc4gc22e75bfd54b"]))

문장: 그런데 그들이 널 반으로 갈라놓지 않았다고?
0.75


In [17]:
# h0005b404g6facg4574g9a8cg6de138cc2fd6
print("문장:", userTranslate_dict["h0005b404g6facg4574g9a8cg6de138cc2fd6"])
print(detect_language(userTranslate_dict["h0005b404g6facg4574g9a8cg6de138cc2fd6"]))

문장: 구속될 수 없습니다.
0.7272727272727273


In [18]:
english_corp = []
korean_corp = []
target_key = []

In [21]:
for key in english_dict:
    if detect_language(userTranslate_dict[key]) >= 0.4:
        korean_corp.append(userTranslate_dict[key])
        english_corp.append(english_dict[key])
    else:
        target_key.append(key)
    

In [29]:
print(len(korean_corp))
print(len(english_corp))

52168
52168


In [31]:
df_dialog_BG3 = pd.DataFrame(columns = ["english", "korean"])

In [32]:
df_dialog_BG3['english'] = english_corp
df_dialog_BG3['korean'] = korean_corp

In [33]:
df_dialog_BG3

,english,korean
0,Replace Spell,주문 교체하기
1,"And if you ain't who you say you are, I reckon...",그리고 네 정체가 너가 말한대로가 아니라면 정당한 공격대상으로 간주하겠어.
2,Target can't be Restrained.,구속될 수 없습니다.
3,"You did just lift a mountain from our heads, s...",넌 방금 우리 머리에서 산을 들어 올려줬으니... 그것만으로도 충분하겠지.
4,Players have different game versions. Please e...,플레이어들의 게임 버전이 일치하지 않습니다. 최신 상태인지 확인하세요!
...,...,...
52163,*The fright-choir fades. A single melody rises...,*무서운 노래가 사그라듭니다. 깊게 울리며 두드러지는 단 하나의 멜로디가 다른 멜로...
52164,I see only fragments. Your mind is confused - ...,파편들밖에 안 보이잖아. 너의 생각은 혼잡한 상태군. 생각을 정리하기 위해 해야 할...
52165,Call forth a beam of light and increase the ch...,대상을 향한 다음 공격이 적중할 확률을 높이는 빛줄기를 발사합니다.
52166,Call forth a beam of light and increase the ch...,대상을 향한 다음 공격이 적중할 확률을 높이는 빛줄기를 발사합니다.


In [35]:
df_dialog_BG3["english"].iloc[52167]

'Call forth a beam of light and increase the chance that the next attack against the target hits.'

In [37]:
df_dialog_BG3.reset_index(inplace=True)

In [38]:
df_dialog_BG3.rename(columns={"index":"num"}, inplace=True)

In [39]:
df_dialog_BG3

,num,english,korean
0,0,Replace Spell,주문 교체하기
1,1,"And if you ain't who you say you are, I reckon...",그리고 네 정체가 너가 말한대로가 아니라면 정당한 공격대상으로 간주하겠어.
2,2,Target can't be Restrained.,구속될 수 없습니다.
3,3,"You did just lift a mountain from our heads, s...",넌 방금 우리 머리에서 산을 들어 올려줬으니... 그것만으로도 충분하겠지.
4,4,Players have different game versions. Please e...,플레이어들의 게임 버전이 일치하지 않습니다. 최신 상태인지 확인하세요!
...,...,...,...
52163,52163,*The fright-choir fades. A single melody rises...,*무서운 노래가 사그라듭니다. 깊게 울리며 두드러지는 단 하나의 멜로디가 다른 멜로...
52164,52164,I see only fragments. Your mind is confused - ...,파편들밖에 안 보이잖아. 너의 생각은 혼잡한 상태군. 생각을 정리하기 위해 해야 할...
52165,52165,Call forth a beam of light and increase the ch...,대상을 향한 다음 공격이 적중할 확률을 높이는 빛줄기를 발사합니다.
52166,52166,Call forth a beam of light and increase the ch...,대상을 향한 다음 공격이 적중할 확률을 높이는 빛줄기를 발사합니다.


In [40]:
df_dialog_set = pd.read_csv("../train_data/dialog_BG2_POE2_BG1.csv", lineterminator='\n')

In [41]:
df_dialog_set

,num,english,korean
0,0,"No, I'm sorry, none of them sound familiar.","아니오, 미안하지만, 그것들 가운데 아는 것은 없소."
1,1,You played Elminster?,엘민스터를 연기했습니까?
2,2,"Uh, the yugoloth, was it? Yeah, you stole the ...","어, 유골로스, 맞습니까? 그래요, 내 기억이 맞다면 당신은 그것으로 쇼를 독차지했었소."
3,3,"And, who knows, we were rehearsing for Picocci...","그리고, 누가 압니까, 우리는 피코키오의 ""에테르 속에서의 사흘간""을 연습하고 있었..."
4,4,"Oh, my dark ravens, let us stop our squawking....","오, 이제 불평은 그만둡시다. 만약 당신이 계속 나와 함께 할 생각이라면, 나는 이..."
...,...,...,...
148429,148429,"A witch, you say? Oh, right. I rescued her. We...","마녀라고 했소? 오, 그렇지. 그녀를 구출했소. 우린 헤어졌지."
148430,148430,(She is gone! Rashemaar fools tramping about t...,"(그녀가 가버렸군! 라셰멘 멍청이들이 렐름을 헤매다니, 테이의 눈을 피할 수 있으리..."
148431,148431,A Ring for a Lady\nI have given Fergus a ring ...,숙녀를 위한 반지\n페르거스에게 숙녀에게 사랑을 바칠 반지를 건내주었다. 행운이 따...
148432,148432,Troubles in the Region: Rumors and Happenings\...,"지역 내 문제들\n내쉬켈 광산의 깊숙한 곳을 파헤쳤으므로, 베런 개스트킬에게 돌아가..."


In [42]:
df_dialog = pd.concat([df_dialog_set, df_dialog_BG3])

In [43]:
df_dialog

,num,english,korean
0,0,"No, I'm sorry, none of them sound familiar.","아니오, 미안하지만, 그것들 가운데 아는 것은 없소."
1,1,You played Elminster?,엘민스터를 연기했습니까?
2,2,"Uh, the yugoloth, was it? Yeah, you stole the ...","어, 유골로스, 맞습니까? 그래요, 내 기억이 맞다면 당신은 그것으로 쇼를 독차지했었소."
3,3,"And, who knows, we were rehearsing for Picocci...","그리고, 누가 압니까, 우리는 피코키오의 ""에테르 속에서의 사흘간""을 연습하고 있었..."
4,4,"Oh, my dark ravens, let us stop our squawking....","오, 이제 불평은 그만둡시다. 만약 당신이 계속 나와 함께 할 생각이라면, 나는 이..."
...,...,...,...
52163,52163,*The fright-choir fades. A single melody rises...,*무서운 노래가 사그라듭니다. 깊게 울리며 두드러지는 단 하나의 멜로디가 다른 멜로...
52164,52164,I see only fragments. Your mind is confused - ...,파편들밖에 안 보이잖아. 너의 생각은 혼잡한 상태군. 생각을 정리하기 위해 해야 할...
52165,52165,Call forth a beam of light and increase the ch...,대상을 향한 다음 공격이 적중할 확률을 높이는 빛줄기를 발사합니다.
52166,52166,Call forth a beam of light and increase the ch...,대상을 향한 다음 공격이 적중할 확률을 높이는 빛줄기를 발사합니다.


In [44]:
df_dialog.reset_index(inplace=True)

In [45]:
df_dialog.drop(["num"], axis=1, inplace=True)
df_dialog.drop(["index"], axis=1, inplace=True)

In [46]:
df_dialog

,english,korean
0,"No, I'm sorry, none of them sound familiar.","아니오, 미안하지만, 그것들 가운데 아는 것은 없소."
1,You played Elminster?,엘민스터를 연기했습니까?
2,"Uh, the yugoloth, was it? Yeah, you stole the ...","어, 유골로스, 맞습니까? 그래요, 내 기억이 맞다면 당신은 그것으로 쇼를 독차지했었소."
3,"And, who knows, we were rehearsing for Picocci...","그리고, 누가 압니까, 우리는 피코키오의 ""에테르 속에서의 사흘간""을 연습하고 있었..."
4,"Oh, my dark ravens, let us stop our squawking....","오, 이제 불평은 그만둡시다. 만약 당신이 계속 나와 함께 할 생각이라면, 나는 이..."
...,...,...
200597,*The fright-choir fades. A single melody rises...,*무서운 노래가 사그라듭니다. 깊게 울리며 두드러지는 단 하나의 멜로디가 다른 멜로...
200598,I see only fragments. Your mind is confused - ...,파편들밖에 안 보이잖아. 너의 생각은 혼잡한 상태군. 생각을 정리하기 위해 해야 할...
200599,Call forth a beam of light and increase the ch...,대상을 향한 다음 공격이 적중할 확률을 높이는 빛줄기를 발사합니다.
200600,Call forth a beam of light and increase the ch...,대상을 향한 다음 공격이 적중할 확률을 높이는 빛줄기를 발사합니다.


In [47]:
df_dialog.reset_index(inplace=True)

In [48]:
df_dialog.rename(columns={"index":"num"}, inplace=True)

In [49]:
df_dialog

,num,english,korean
0,0,"No, I'm sorry, none of them sound familiar.","아니오, 미안하지만, 그것들 가운데 아는 것은 없소."
1,1,You played Elminster?,엘민스터를 연기했습니까?
2,2,"Uh, the yugoloth, was it? Yeah, you stole the ...","어, 유골로스, 맞습니까? 그래요, 내 기억이 맞다면 당신은 그것으로 쇼를 독차지했었소."
3,3,"And, who knows, we were rehearsing for Picocci...","그리고, 누가 압니까, 우리는 피코키오의 ""에테르 속에서의 사흘간""을 연습하고 있었..."
4,4,"Oh, my dark ravens, let us stop our squawking....","오, 이제 불평은 그만둡시다. 만약 당신이 계속 나와 함께 할 생각이라면, 나는 이..."
...,...,...,...
200597,200597,*The fright-choir fades. A single melody rises...,*무서운 노래가 사그라듭니다. 깊게 울리며 두드러지는 단 하나의 멜로디가 다른 멜로...
200598,200598,I see only fragments. Your mind is confused - ...,파편들밖에 안 보이잖아. 너의 생각은 혼잡한 상태군. 생각을 정리하기 위해 해야 할...
200599,200599,Call forth a beam of light and increase the ch...,대상을 향한 다음 공격이 적중할 확률을 높이는 빛줄기를 발사합니다.
200600,200600,Call forth a beam of light and increase the ch...,대상을 향한 다음 공격이 적중할 확률을 높이는 빛줄기를 발사합니다.


In [50]:
# df_dialog.to_csv(r"../train_data/dialog_BG2_POE2_BG1_BG3.csv", index=False, encoding='utf-8')

In [52]:
# from datasets import Dataset, DatasetDict

# datasets_train = DatasetDict({
#     "train": Dataset.from_pandas(df_dialog)
# })

In [53]:
# datasets_train

DatasetDict({
    train: Dataset({
        features: ['num', 'english', 'korean'],
        num_rows: 200602
    })
})

In [54]:
# from huggingface_hub import login

# login(token= input("Token: "))

# datasets_train.push_to_hub("aeolian83/DnD_translate_v2.0")

Token will not been saved to git credential helper. Pass `add_to_git_credential=True` if you want to set the git credential as well.
Token is valid (permission: write).
Your token has been saved to /home/aeolian83/.cache/huggingface/token
Login successful


Pushing dataset shards to the dataset hub:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/201 [00:00<?, ?ba/s]